In [1]:
import json
from pymongo import MongoClient
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [2]:
# === Configuración ===
json_file_path = "dataset/arxiv-metadata-oai-snapshot.json"
batch_size = 100
max_workers = 6  # Ajusta según tu CPU y MongoDB

In [3]:
# === Conectar a MongoDB ===
client = MongoClient(
    'mongodb://localhost:30001,localhost:30002,localhost:30003/?replicaSet=my-replica-set'
)
db = client["arxiv_db"]
collection = db["articles"]

In [4]:
# === Función para insertar lote ===
def insert_batch(batch):
    try:
        collection.insert_many(batch, ordered=False)
    except Exception as e:
        print("Error al insertar batch:", e)

In [5]:
# === Contar líneas para tqdm ===
def count_lines(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return sum(1 for _ in f)

total_lines = count_lines(json_file_path)

In [ ]:
# === Lectura + carga paralela ===
with open(json_file_path, 'r', encoding='utf-8') as f, tqdm(total=total_lines, desc="Cargando") as pbar:
    batch = []
    futures = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for line in f:
            try:
                record = json.loads(line)
                # 👉 Agregar campo pdf_source
                record["pdf_source"] = f"https://arxiv.org/pdf/{record['id']}"
                batch.append(record)
                if len(batch) >= batch_size:
                    future = executor.submit(insert_batch, batch)
                    futures.append(future)
                    batch = []
            except json.JSONDecodeError:
                continue
            pbar.update(1)

        # Último batch
        if batch:
            futures.append(executor.submit(insert_batch, batch))
            pbar.update(len(batch))

# === Esperar a que terminen todas las cargas ===
for future in futures:
    future.result()
print("✅ Carga paralela completa.")

Cargando:   0%|          | 0/2735264 [00:00<?, ?it/s]

Cargando:  49%|████▉     | 1339189/2735264 [00:30<00:34, 40512.90it/s]

Error al insertar batch: localhost:30001: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),localhost:30002: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),localhost:30003: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 684a48bec1589210060083bb, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('localhost', 30001) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:30001: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expres

Cargando:  90%|█████████ | 2471433/2735264 [01:00<00:09, 26477.53it/s]

Error al insertar batch: localhost:30001: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),localhost:30002: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),localhost:30003: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 684a48bec1589210060083bb, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('localhost', 30001) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:30001: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expres

Cargando: 2735328it [01:20, 92062.51it/s]                             

In [13]:
'''
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27018/")
db = client["arxiv_db"]
collection = db["articles"]

# ⚠️ Eliminar todos los documentos
collection.delete_many({})
print("🗑️ Todos los documentos eliminados.")
'''

🗑️ Todos los documentos eliminados.
